In [1]:
import re
from collections import Counter


In [2]:
def words(text): return re.findall(r'\w+', text.lower())

sherlock = Counter(words(open('/kaggle/input/a2data1/sherlock.txt').read()))
testset1 = Counter(words(open('/kaggle/input/a2data1/test.txt').read()))
aspell = Counter(words(open('/kaggle/input/spelling/aspell.txt').read()))
big = Counter(words(open('/kaggle/input/spelling/big.txt').read()))
birkbeck = Counter(words(open('/kaggle/input/spelling/birkbeck.txt').read()))
testset2 = Counter(words(open('/kaggle/input/spelling/spell-testset1.txt').read()))
testset3 = Counter(words(open('/kaggle/input/spelling/spell-testset2.txt').read()))
wikipedia = Counter(words(open('/kaggle/input/spelling/wikipedia.txt').read()))
WORDS = sherlock  + aspell + big + birkbeck +  wikipedia
TEST = testset1 + testset2 + testset3


In [3]:
def P(word, N=sum(WORDS.values())): 
    #"Probability of `word`."
    return WORDS[word] / N


In [4]:
def correction(word):
    # Generate possible corrections
    candidates_list = candidates(word)

    # Calculate probability for each candidate
    probabilities = {candidate: P(candidate) for candidate in candidates_list}

    # Return the correction with the highest probability
    return max(probabilities, key=probabilities.get)


In [5]:
def candidates(word):
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])


In [6]:
def known(words): 
    ## "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in WORDS)


In [7]:
def edits1(word):
    #"All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)


In [8]:
def edits2(word): 
    #"All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [9]:
print(correction('corrrectud'))

corrected


In [10]:
def run_tests():
    # Test correction function
    test_correction('korrectud', 'corrected')

In [11]:
def test_correction(input_word, expected_correction):
    result = correction(input_word)
    assert result == expected_correction, f"Expected: {expected_correction}, Got: {result}"
    print(f"Test Passed for '{input_word}': {result}")


In [12]:
run_tests()

Test Passed for 'korrectud': corrected


In [13]:
corrected_text=[]
for input_word in TEST:
        corrected_word = correction(input_word)
        corrected_text.append(corrected_word)
        
with open("corrected_test", "w") as file:
    # Write each element of the list to the file
    for input_word in TEST:
        file.write(str(correction(input_word)) + " ")
